In [ ]:
import os
import json

import boto3
import sagemaker
from sagemaker.local import LocalSession
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session

In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
tuning_job_name = 'pytorch-training-220627-1308'

In [ ]:
sagemaker_client = boto3.client('sagemaker')
s3_client = boto3.client("s3")
sagemaker_runtime_client = boto3.client('runtime.sagemaker')

In [ ]:
tunning_job = sagemaker_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)
print(tunning_job['TrainingJobStatusCounters'])
tunning_job['BestTrainingJob']

In [ ]:
best_model = sagemaker_client.describe_training_job(TrainingJobName=tunning_job['BestTrainingJob']['TrainingJobName'])
best_model['ModelArtifacts']['S3ModelArtifacts']

In [ ]:
model = PyTorchModel(
    entry_point="inference.py",
    source_dir="script",
    role=role,
    model_data=best_model['ModelArtifacts']['S3ModelArtifacts'],
    framework_version="1.10.0",
    py_version="py38",
    sagemaker_session=LocalSession()
)

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    # instance_type='ml.p3.2xlarge'
    instance_type='local'
)

In [ ]:
test = {
    "inputs": "teste"
}

In [ ]:
predictor.predict(test)

In [ ]:
sagemaker_runtime_client.invoke_endpoint(
    EndpointName='pytorch-inference-2022-06-27-22-42-32-433',
    Body=test,
    ContentType='application/json'
)